In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import warnings
import pandas as pd
import copy
import timeit
from dotenv import load_dotenv

warnings.filterwarnings("ignore")

## Import

In [2]:
from app.utils.data_loader import DataLoader
from app.utils.health_score import HealthScore
from app.utils.portfolios import Portfolios
from app.utils.portfolios_repo import PortfoliosRepository
from app.utils.portfolios_service import PortfolioService
from app.utils.portprop_matrices import PortpropMatrices
from app.utils.portprop_matrices_repo import PortpropMatricesRepository
from app.utils.rebalancer import Rebalancer
from app.utils.rebalancer_repo import RebalancerRepository

## Load Data

In [3]:
load_dotenv()  # Load environment variables from .env file
print(os.getenv("LOAD_DATA_FROM_DWH"))

False


In [4]:
data_loader_dwh = DataLoader(load_from_db=True)
data_loader_parquet = DataLoader(load_from_db=False)

In [5]:
ppm_repo = PortpropMatricesRepository(data_loader=data_loader_parquet)
ports_repo = PortfoliosRepository(data_loader=data_loader_parquet)
ports_repo_dwh = PortfoliosRepository(data_loader=data_loader_dwh)
rebalancer_repo = RebalancerRepository(data_loader=data_loader_parquet)

In [6]:
client_out_enriched = ports_repo.load_client_out_product_enriched(
    as_of_date="2025-10-31"
)

In [8]:
client_styles = ports_repo.load_client_style(as_of_date="2025-10-31")

query


In [9]:
ports_ref_table = {
    "product_mapping": ports_repo.load_product_mapping('2025-09-30'),
    "product_underlying": ports_repo.load_product_underlying(),
}

In [10]:
ppm_ref_dict = {
    "portprop_factsheet": ppm_repo.load_portprop_factsheet(),
    "portprop_benchmark": ppm_repo.load_portprop_benchmark(),
    "portprop_ge_mapping": ppm_repo.load_portprop_ge_mapping(),
    "portprop_fallback": ppm_repo.load_portprop_fallback(),
    "portprop_ret_eow": ppm_repo.load_portprop_ret_eow(),
    "advisory_health_score": ppm_repo.load_advisory_health_score(),
}

In [11]:
rb_ref_dict = {
    "es_sell_list": rebalancer_repo.load_es_sell_list(),
    "product_recommendation_rank_raw": rebalancer_repo.load_product_recommendation_rank_raw(),
    "mandate_allocation": rebalancer_repo.load_mandate_candidates(),
}

In [12]:
acct_client = ports_repo.load_acct_customer_mapping()

## Instances

### map sec id product_mapping

In [13]:
prod_comp_keys = ['product_id', 'src_sharecodes', 'desk', 'port_type', 'currency']

In [14]:
ports_ref_table['product_mapping'].reset_index(inplace=True)
ports_ref_table['product_mapping'].rename(columns={'index': 'sec_id'}, inplace=True)

### map sec id mandate_allocation

In [15]:
rb_ref_dict['mandate_allocation'] = rb_ref_dict['mandate_allocation'].merge(
    ports_ref_table['product_mapping'][prod_comp_keys + ['sec_id']],
    on=prod_comp_keys,
    how='left',
    validate='one_to_one'
)

### map sec id product_recommendation_rank_raw

In [16]:
mf_prod_mapping_rank = ports_ref_table['product_mapping'][ports_ref_table['product_mapping']['product_type_desc'] == 'Mutual Fund']

In [17]:
product_recommendation_rank_raw = rb_ref_dict['product_recommendation_rank_raw'][["src_sharecodes", "desk", "currency"] + ['is_ui',
       'rank_product']]


rb_ref_dict['product_recommendation_rank_raw'] = product_recommendation_rank_raw.merge(
    mf_prod_mapping_rank,
    on=["src_sharecodes", "desk", "currency"],
    how='left',
    validate='one_to_one'
)

### sec id product mappping df_out_enriched

In [18]:
client_out_enriched = client_out_enriched.merge(
    ports_ref_table['product_mapping'][prod_comp_keys + ['sec_id']],
    on=prod_comp_keys,
    how='left',
    validate='many_to_one'
)

### Check Mapping must have sec_id

In [19]:
for name, df in [
    ("ports_ref_table['product_mapping']", ports_ref_table['product_mapping']),
    ("client_out_enriched", client_out_enriched),
    ("rb_ref_dict['product_recommendation_rank_raw']", rb_ref_dict['product_recommendation_rank_raw']),
    ("rb_ref_dict['mandate_allocation']", rb_ref_dict['mandate_allocation']),
]:
    print(f"{name} has 'sec_id'? {'sec_id' in df.columns}")

ports_ref_table['product_mapping'] has 'sec_id'? True
client_out_enriched has 'sec_id'? True
rb_ref_dict['product_recommendation_rank_raw'] has 'sec_id'? True
rb_ref_dict['mandate_allocation'] has 'sec_id'? True


### Initialized

In [21]:
## Portsfolios
ports_all = Portfolios()
ports_all.set_ref_tables(ports_ref_table)
df_out, df_style, port_ids, port_id_mapping = ports_all.create_portfolio_id(
    client_out_enriched, client_styles, column_mapping=["as_of_date", "customer_id"]
)
ports_all.set_portfolio(df_out, df_style, port_ids, port_id_mapping)

## Portfolio Service
port_service = PortfolioService(ports_all)

## Portprop Matrices
ppm = PortpropMatrices(ppm_ref_dict)

## Health Score
hs = HealthScore()

## Rebalancer
rb = Rebalancer(
    client_investment_style="Moderate High Risk",
    client_classification="UI",
    discretionary_acceptance=0.2,
    new_money=1_000_000,
    product_whitelist=["KKP", "PTTEP"],
    product_blacklist=["KKP GNP", "K-GSELECTU-A(A)"],
)
rb.set_ref_tables(rb_ref_dict)

## Portfolio (Service)

In [22]:
## get single port from customer id
port = port_service.get_client_portfolio(customer_id=12560)

In [23]:
port.df_style.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 37420 to 37420
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   port_id                        1 non-null      int64 
 1   client_full_name_th            1 non-null      string
 2   client_first_name_en           1 non-null      string
 3   client_last_name_en            1 non-null      string
 4   port_investment_style          1 non-null      string
 5   client_tier                    1 non-null      string
 6   business_unit                  1 non-null      string
 7   client_segment_by_inv_aum      1 non-null      string
 8   client_sub_segment_by_inv_aum  1 non-null      string
 9   sales_id                       1 non-null      string
 10  ui_client                      1 non-null      string
 11  sales_first_name_en            1 non-null      string
 12  sales_team                     1 non-null      string
 13  portpo

In [24]:
## get port outstanding
port.df_out

,customer_id,as_of_date,product_id,src_sharecodes,desk,port_type,currency,product_display_name,product_type_desc,asset_class_name,...,expected_return,es_core_port,es_sell_list,flag_top_pick,flag_tax_saving,value,sec_id,port_id,asset_class_code,weight
208873,12560,2025-10-31,M00211737,KTPCRED-UI-R,TRADE,L,THB,KTPCRED-UI-R,Mutual Fund,Global Equity,...,0.077,False,<NA>,Not Top-Pick,<NA>,4.848189e+06,180992.0,1147,AA_GE,0.028842
208908,12560,2025-10-31,S00271934,SAIN:xlon,SAXO,L,GBP,The Scottish American Investment Company PLC,Listed Securities,Global Equity,...,0.077,False,<NA>,Not Top-Pick,<NA>,3.453405e+05,139415.0,1147,AA_GE,0.002054
209622,12560,2025-10-31,C00144524,SS_2000178882,TRADE,L,THB,KKP Smart Settlement (Individual),Cash,Cash and Cash Equivalent,...,0.040,False,<NA>,Not Top-Pick,<NA>,2.235003e+05,307229.0,1147,AA_CASH,0.001330
209624,12560,2025-10-31,C00172837,SS_2013980702,GIS,L,USD,USD Saving Account,Cash,Cash and Cash Equivalent,...,0.048,False,<NA>,Not Top-Pick,<NA>,1.426275e+04,191374.0,1147,AA_CASH,0.000085
209698,12560,2025-10-31,M00216727,KT-GOLDUH-A,TRADE,L,THB,KT-GOLDUH-A,Mutual Fund,Alternative,...,0.043,False,<NA>,Top-Pick,<NA>,3.433835e+05,275126.0,1147,AA_ALT,0.002043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226613,12560,2025-10-31,B00214015,TMN266A,TRADE,L,THB,TMN266A,Fixed Income,Fixed Income,...,0.022,False,<NA>,Not Top-Pick,<NA>,1.015599e+07,168956.0,1147,AA_FI,0.060419
226668,12560,2025-10-31,M00162119,KKP PGE-UH,TRADE,L,THB,KKP PGE-UH,Mutual Fund,Global Equity,...,0.077,False,<NA>,Top-Pick,<NA>,1.350555e+07,292998.0,1147,AA_GE,0.080346
226690,12560,2025-10-31,M00250758,LU2405385126,GIS,L,USD,"GS JAPAN EQUITY PARTNERS PORTFOLIO ""OCS"" (USDH...",Mutual Fund,Global Equity,...,0.077,False,<NA>,Top-Pick,<NA>,1.890884e+05,268758.0,1147,AA_GE,0.001125
226695,12560,2025-10-31,P00156097,ODL,GIS,L,USD,ODL,Private Market,Global Equity,...,0.077,False,<NA>,Not Top-Pick,<NA>,6.491295e+06,128335.0,1147,AA_GE,0.038617


In [25]:
ppm.df_port_fs

,symbol,bm_name,asset_class,weight
0,KKPEQTHB,TMBMONEY,Cash and Cash Equivalent,0.01
1,KKPFITHB,SETPREIT50% + FSTREI50%,Alternative,0.00
2,KKPFITHB,BB Barclay Global Aggregate Hedged-USD (70%) ...,Fixed Income,0.00
3,KKPFITHB,Bloomberg Commodity TR,Alternative,0.00
4,KKPFITHB,Dow Jones Brookfield Global Infrastructure Ind...,Alternative,0.00
...,...,...,...,...
1153,KKPEQTHB,SET TRI,Local Equity,0.00
1154,KKPEQTHB,SETPF&REIT,Alternative,0.00
1155,KKPEQTHB,S&P Global REIT Index USD (TR),Alternative,0.00
1156,KKPEQTHB,"All Thai Gov Bond, Avg. Dur. 7.79 (25/09/23), ...",Fixed Income,0.00


In [26]:
# get port allocation lookthrough
port.get_portfolio_asset_allocation_lookthrough(ppm)

asset_class,port_id,aa_alt,aa_cash,aa_fi,aa_ge,aa_le
0,1147,0.081543,0.030532,0.386234,0.397263,0.104428


In [27]:
# get model allocation
port.get_model_asset_allocation_lookthrough(ppm)

,port_id,port_investment_style,portpop_styles,aa_alt_model,aa_cash_model,aa_fi_model,aa_ge_model,aa_le_model
0,1147,Moderate High Risk,Medium to Moderate High Risk,0.13,0.06,0.27,0.45,0.05


In [28]:
port.df_style

,port_id,client_full_name_th,client_first_name_en,client_last_name_en,port_investment_style,client_tier,business_unit,client_segment_by_inv_aum,client_sub_segment_by_inv_aum,sales_id,ui_client,sales_first_name_en,sales_team,portpop_styles
37420,1147,สุภจ*************,SUPH****,NGA****,Moderate High Risk,Wealth,PWM,Qualified,Large,323,Y,Nara*****,Rung********,Medium to Moderate High Risk


## Healthscore

In [31]:
## get client health score
health_score, health_score_comp = port.get_portfolio_health_score(ppm, hs)

In [32]:
health_score

,port_id,expected_return,expected_return_model,score_ret,volatility,volatility_model,score_vol,score_portfolio_risk,acd,score_acd,ged,score_ged,score_diversification,score_bulk_risk,score_issuer_risk,score_non_cover_global_stock,score_non_cover_local_stock,score_non_cover_mutual_fund,score_not_monitored_product,health_score
0,1147,0.051034,0.06513,-1,0.081996,0.1015,0,-1,0.149902,0,0.060729,0,0,0,0,0,-1,0,-1.0,8.0


In [33]:
health_score_comp.head()

,port_id,sec_id,product_display_name,product_type_desc,asset_class_name,value,weight,aa_alt,aa_cash,aa_fi,...,ge_other,expected_return,volatility,is_bulk_risk,underlying_company,issure_risk_group,coverage_prdtype,score_non_cover_global_stock,score_non_cover_local_stock,score_non_cover_mutual_fund
0,1147,180992.0,KTPCRED-UI-R,Mutual Fund,Global Equity,4.848189e+06,0.028842,0.000000,0.000000,0.0,...,0.004937,0.002221,4.750711e-03,False,<NA>,NaN,MUTUAL_FUND,0,0,0
1,1147,139415.0,The Scottish American Investment Company PLC,Listed Securities,Global Equity,3.453405e+05,0.002054,0.000000,0.000000,0.0,...,0.000352,0.000158,3.383971e-04,False,<NA>,NaN,GLOBAL_STOCK,0,0,0
2,1147,307229.0,KKP Smart Settlement (Individual),Cash,Cash and Cash Equivalent,2.235003e+05,0.001330,0.000000,0.001330,0.0,...,NaN,0.000053,-3.427948e-08,False,<NA>,NaN,N/A,0,0,0
3,1147,191374.0,USD Saving Account,Cash,Cash and Cash Equivalent,1.426275e+04,0.000085,0.000000,0.000085,0.0,...,NaN,0.000004,-2.500736e-08,False,<NA>,NaN,N/A,0,0,0
4,1147,275126.0,KT-GOLDUH-A,Mutual Fund,Alternative,3.433835e+05,0.002043,0.002043,0.000000,0.0,...,NaN,0.000088,5.679999e-05,False,<NA>,NaN,MUTUAL_FUND,0,0,0


In [34]:
health_score_comp.columns

Index(['port_id', 'sec_id', 'product_display_name', 'product_type_desc',
       'asset_class_name', 'value', 'weight', 'aa_alt', 'aa_cash', 'aa_fi',
       'aa_ge', 'aa_le', 'ge_em', 'ge_eur', 'ge_jp', 'ge_us', 'ge_other',
       'expected_return', 'volatility', 'is_bulk_risk', 'underlying_company',
       'issure_risk_group', 'coverage_prdtype', 'score_non_cover_global_stock',
       'score_non_cover_local_stock', 'score_non_cover_mutual_fund'],
      dtype='object')

## Rebalancer

In [35]:
rb.rebalance(port, ppm, hs)

(<app.utils.portfolios.Portfolios at 0x7db4f1459ee0>,
   transaction_no batch_no port_id  sec_id    product_id  src_sharecodes  \
 0              1        1    1147  317393  CTHB00000000  Cash Proxy THB   
 1              2        2    1147  276307     S00087551          BTSGIF   
 2              3        2    1147  317393  CTHB00000000  Cash Proxy THB   
 3              4        3    1147   54161     S00237377           VAYU1   
 4              5        3    1147  317393  CTHB00000000  Cash Proxy THB   
 5              6        4    1147  157048  DTHB00000000        KKPBATHB   
 6              7        4    1147  317393  CTHB00000000  Cash Proxy THB   
 
     desk port_type currency product_display_name  product_type_desc  \
 0  TRADE         L      THB       Cash Proxy THB               Cash   
 1  TRADE         L      THB               BTSGIF  Listed Securities   
 2  TRADE         L      THB       Cash Proxy THB               Cash   
 3  TRADE         L      THB                VAYU